In [10]:
from mp_api.client import routes
from pymatgen.core.periodic_table import Element

In [11]:
API_KEY = "fqPPo7Czb5mkbFh8mltlZd0I33csuKv0"
bpr = routes.ElectrodeRester(API_KEY)

In [7]:
# version of MP database
mpr.get_database_version()

'2023.11.1'

In [9]:
# 根据chemsys_list来查询mp中相关的所有结构
# docs=mpr.summary.search(chemsys=chemsys_list, all_fields=False,  num_sites=(40, 100), energy_above_hull=(0, 0.1), fields=["material_id", "formula_pretty", "nsites", "symmetry", "formation_energy_per_atom", "is_stable"])

Retrieving SummaryDoc documents:   0%|          | 0/852 [00:00<?, ?it/s]

In [ ]:
docs = bpr.search(working_ion=Element('Li'), all_fields=True)

Retrieving InsertionElectrodeDoc documents:   0%|          | 0/2440 [00:00<?, ?it/s]

In [17]:
docs[0]

MPDataDoc<InsertionElectrodeDoc>(
battery_id='mp-753047_Li',
battery_formula='Li1-2CoNiO4',
working_ion=Element Li,
num_steps=1,
max_voltage_step=0.0,
last_updated=datetime.datetime(2022, 6, 28, 3, 50, 55, 322000),
framework=Comp: Co1 Ni1 O4,
framework_formula='CoNiO4',
elements=[Element Co, Element Ni, Element O],
nelements=3,
chemsys='Co-Ni-O',
formula_anonymous='ABC4',
warnings=[],
formula_charge='LiCoNiO4',
formula_discharge='Li2CoNiO4',
max_delta_volume=0.04218560461693888,
average_voltage=3.371941503333329,
capacity_grav=137.08763113805108,
capacity_vol=653.5092993789779,
energy_grav=462.2514730280448,
energy_vol=2203.5951293902613,
fracA_charge=0.14285714285714285,
fracA_discharge=0.25,
stability_charge=0.0,
stability_discharge=0.017019385624999828,
id_charge=MPID(mp-754509),
id_discharge=MPID(mp-753536),
host_structure=Structure Summary
Lattice
    abc : 5.692275219521804 5.692275219521804 5.692275219521804
 angles : 119.95572684169043 119.7685878701071 90.23899282249465
 volum

In [18]:
formula_battery = []
id_battery = []
id_charge = []
id_discharge = []
for i in range(len(docs)):
    id_battery.append(docs[i].battery_id)
    formula_battery.append(docs[i].battery_formula)
    id_charge.append(docs[i].id_charge)
    id_discharge.append(docs[i].id_discharge)

In [25]:
battery_data = pd.DataFrame()
battery_data['battery_formula'] = formula_battery
battery_data['battery_id'] = id_battery
battery_data['charge_id'] = id_charge
battery_data['discharge_id'] = id_discharge
battery_data

,battery_formula,battery_id,charge_id,discharge_id
0,Li1-2CoNiO4,mp-753047_Li,mp-754509,mp-753536
1,Li0-1In2CuSe4,mp-1078168_Li,mp-1078168,mp-1079491
2,Li2-4CoNi3O8,mp-757871_Li,mp-1178027,mp-867640
3,Li0-4Fe3Cu(PO4)4,mp-775191_Li,mp-775191,mp-1177485
4,Li0-1FeCuS2,mp-753211_Li,mp-753211,mp-755288
...,...,...,...,...
2435,Li0-1NiP2O7,mp-26785_Li,mp-26786,mp-26785
2436,Li0-3Ni3(PO4)4,mp-504236_Li,mp-1101533,mp-504236
2437,Li1-2NiP2O7,mp-758298_Li,mp-766087,mp-758298
2438,Li0-2Ni(PO3)4,mp-26195_Li,mp-504269,mp-26195


In [27]:
battery_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2440 entries, 0 to 2439
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   battery_formula  2440 non-null   object
 1   battery_id       2440 non-null   object
 2   charge_id        2440 non-null   object
 3   discharge_id     2440 non-null   object
dtypes: object(4)
memory usage: 76.4+ KB


In [26]:
df = pd.DataFrame(columns=['material_id', 'formula_pretty', 'nsites', 'crystal_system', 'formation_energy_per_atom', 'is_stable'])
for i in docs:
    data=[i.material_id, i.formula_pretty, i.nsites, i.symmetry.symbol, i.formation_energy_per_atom, i.is_stable]
    df.loc[len(df)]=data

In [27]:
df[:10]

,material_id,formula_pretty,nsites,crystal_system,formation_energy_per_atom,is_stable
0,mp-1178332,Li(CoO2)2,56,P1,-1.457210,False
1,mp-1377260,Li(CoO2)2,56,P4_322,-1.457917,False
2,mp-25383,Li(CoO2)2,56,Fd-3m,-1.451605,False
3,mp-773076,Li(CoO2)2,56,P4_332,-1.486767,False
4,mp-774082,Li(CoO2)2,56,P2_1,-1.504020,True
5,mp-530277,Li10Co11O21,84,P-1,-1.694866,False
6,mp-1185179,Li10Co4O9,46,P4_2/nmc,-1.773020,False
7,mp-773128,Li10Co4O9,46,P-1,-1.747897,False
8,mp-769567,Li10Ti11Cr9O40,70,P-1,-2.921830,False
9,mp-769483,Li11(CoO4)2,42,C222_1,-1.911476,False


In [28]:
df.to_csv('./materials.csv')

In [20]:
# 去掉重复值
# i = 0 # 全索引
# m = 0 # 组索引
# while i < 852:
#     n = 0
#     while df.iloc[m].formula_pretty == df.iloc[m+1].formula_pretty:
#         if df.iloc[m].crystal_system == df.iloc[m+1].crystal_system:
#             if df.iloc[m].formation_energy_per_atom > df.iloc[m+1].formation_energy_per_atom:
#                 df.drop(m+1)
#                 n += 1
#             elif df.iloc[m].formation_energy_per_atom < df.iloc[m+1].formation_energy_per_atom:
#                 df.drop(m)
#                 n += 1
#     print(i, n, m)
#     i += n
#     m += 1 
a = 

---------------------------------------------------------------------------
KeyboardInterrupt                         Traceback (most recent call last)
<ipython-input-20-673dc008488a> in <module>
      5     n = 0
      6     while df.iloc[m].formula_pretty == df.iloc[m+1].formula_pretty:
----> 7         if df.iloc[m].crystal_system == df.iloc[m+1].crystal_system:
      8             if df.iloc[m].formation_energy_per_atom > df.iloc[m+1].formation_energy_per_atom:
      9                 df.drop(m+1)

D:\python\conda\lib\site-packages\pandas\core\indexing.py in __getitem__(self, key)
   1101 
   1102             maybe_callable = com.apply_if_callable(key, self.obj)
-> 1103             return self._getitem_axis(maybe_callable, axis=axis)
   1104 
   1105     def _is_scalar_access(self, key: tuple):

D:\python\conda\lib\site-packages\pandas\core\indexing.py in _getitem_axis(self, key, axis)
   1656             self._validate_integer(key, axis)
   1657 
-> 1658             return self.obj